In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import anndata
import numpy as np
adata = anndata.read_h5ad("../../data/raw/bowel_disease.h5ad")

OSError: Unable to synchronously open file (truncated file: eof = 28049408, sblock->base_addr = 0, stored_eof = 2988835887)

In [6]:
# get the path of the project directory
import os
from pathlib import Path
# Get the path of the parent directory
project_dir = str(Path(os.getcwd()).parent.parent)

In [14]:
data_dir = f"{project_dir}/data/raw"

In [15]:
import scanpy as sc
from mmcontext.pp.utils import add_ensembl_ids
if "ensemble_ids" not in adata.var.columns:
    add_ensembl_ids(adata)
if "n_counts" not in adata.obs.columns:
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.obs["n_counts"] = adata.obs.total_counts
adata.write_h5ad(f"{data_dir}/bowel_disease.h5ad")

KeyboardInterrupt: 

In [11]:
geneformer_version = "gc95M"
if geneformer_version == "gc95M":
    token_dictionary_file = f"{project_dir}/../Geneformer/geneformer/token_dictionary_gc95M.pkl"
    gene_mapping_file = f"{project_dir}/../Geneformer/geneformer/ensembl_mapping_dict_gc95M.pkl"
    model_dir = f"{project_dir}/../Geneformer/gf-12L-95M-i4096"

In [13]:
from geneformer import TranscriptomeTokenizer

tk = TranscriptomeTokenizer(
    {"cluster_label": "cluster_label"},
    nproc=16,
    token_dictionary_file=token_dictionary_file,
    gene_mapping_file=gene_mapping_file,
)
tk.tokenize_data(
    data_dir,  # This has to point to the directory not a file
    f"{project_dir}/data/geneformer_datasets",
    "bowel_disease",
    file_format="h5ad",
)

/h/menger/git/mmcontext/.venv/lib/python3.10/site-packages/geneformer/evaluation_utils.py:194: DeprecationWarning: invalid escape sequence '\p'
  label = f"{model_name} (AUC {roc_auc:0.2f} $\pm$ {roc_auc_sd:0.2f})"


NameError: name 'data_dir' is not defined

In [19]:
from geneformer import EmbExtractor

# Or try again with the code from geneformer
extractor = EmbExtractor(
    model_type="Pretrained",
    num_classes=0,
    emb_mode="cls",
    cell_emb_style="mean_pool",
    gene_emb_style="mean_pool",
    filter_data=None,
    max_ncells=1000,
    emb_layer=-1,
    emb_label=None,
    labels_to_plot=None,
    forward_batch_size=100,
    nproc=4,
    summary_stat=None,
    token_dictionary_file=None,
)

In [20]:
embs = extractor.extract_embs(
    "/Users/mengerj/repos/Geneformer/gf-12L-95M-i4096",
    "/Users/mengerj/repos/mmcontext/data/geneformer_datasets/bowel_disease.dataset",
    "../../../outputs/",
    output_prefix="bowel_disease2",
    output_torch_embs=True,
    cell_state=None,
)

  0%|          | 0/10 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


RuntimeError: Placeholder storage has not been allocated on MPS device!